In [190]:

import pandas as pd
import numpy as np
import math
from scipy.spatial import distance
import scipy

In [173]:
import math
import numpy as np

class kdTree_node:
    
    def __init__(self, x, y, split_along_x=True,height=0,nodeId=0):
        self.x = x
        self.y = y
        self.xmax = math.inf
        self.ymax = math.inf
        self.xmin = -math.inf
        self.ymin = -math.inf
        self.split_along_x = split_along_x
        self.left = None
        self.right = None
        self.points = []
        self.height = height
        self.id = nodeId
        
    def __str__(self):
        return "(x="+str(self.x)+",y="+str(self.y)+")"



class kdTree:
    def __init__(self, xs, ys,h):
        
        i_x_sort = np.argsort(xs)
        i_y_sort = np.argsort(ys)
        self.h = h
        self.nodeId = 0
        self.root = self.__buildTree(xs, ys, i_x_sort, i_y_sort, True,None,0)
        
        
    def print(self):
        self.__printSubtree(self.root)
        
    def __printSubtree(self, node):
        if node.left!=None:
            self.__printSubtree(node.left)
        print(node)
        if node.right!=None:
            self.__printSubtree(node.right)
    
    def __select(self, isorted, isecond):
        iy = np.array([]).astype(int)
        for i in isecond:
            r = (isorted==i)
            if r.any()==True:
                iy=np.append(iy,i)
        return iy
    
    def __buildTree(self, xs, ys, ix, iy, splitx=None, father=None,height=0):
        l = ix.shape[0]
        med = l//2
        
        if height == self.h:
            return None
        # Split along the xaxis
        if splitx:
            self.nodeId  = self.nodeId + 1
            n = kdTree_node(xs[ix[med]], ys[ix[med]], True,height,self.nodeId)
            if father != None:
                
                n.xmin = father.xmin
                n.xmax = father.xmax
                n.ymin = father.ymin
                n.ymax = father.ymax
                
                if n.y <= father.y:
                    n.ymax = father.y
                    
                else:
                    n.ymin = father.y 
            

            
            
            for i in range(0,len(xs)):
                if xs[i] >= n.xmin and xs[i] <= n.xmax and  ys[i] >= n.ymin and xs[i] <= n.ymax:
                    n.points.append([xs[i],ys[i]])
            
            if med > 0:
                sub_iy = self.__select(ix[:med],iy)
                n.left = self.__buildTree(xs, ys, ix[:med], sub_iy, False, n,n.height+1)
            if med+1<l:
                sub_iy = self.__select(ix[med+1:], iy)
                n.right = self.__buildTree(xs, ys, ix[med+1:], sub_iy, False, n,n.height+1)
            
        # This node corresponds to a split of the data along y
        else:
            self.nodeId  = self.nodeId + 1
            n = kdTree_node(xs[iy[med]], ys[iy[med]], False,height,self.nodeId)
            if father != None:

                n.xmin = father.xmin
                n.xmax = father.xmax
                n.ymin = father.ymin
                n.ymax = father.ymax

                if n.x < father.x:
                    n.xmax = father.x

                else:
                    n.xmin = father.x


            
            for i in range(0,len(xs)):
                if xs[i] >= n.xmin and xs[i] <= n.xmax and  ys[i] >= n.ymin and xs[i] <= n.ymax:
                    n.points.append([xs[i],ys[i]])
                    
                    
            if med > 0:
                sub_ix = self.__select(iy[:med],ix)
                n.left = self.__buildTree(xs, ys, sub_ix, iy[:med], True, n,n.height+1)
            if med+1<l:
                sub_ix = self.__select(iy[med+1:], ix)
                n.right = self.__buildTree(xs, ys, sub_ix, iy[med+1:], True, n,n.height+1)
                
       
        
        return n 
    
    def is_fully_contained(self, node, r):
        if node:
            if r['xmin'] <= node.xmin and r['xmax'] >= node.xmax and r['ymin'] <= node.ymin and r['ymax'] >= node.ymax:
                return True
        return False
    
    def is_intersect(self, node, r):
        if node:
            if r['ymin'] > node.ymax or r['ymax'] < node.ymin or r['xmin'] > node.xmax or r['xmax'] < node.xmin:
                return False
        return True
    
    def range_search(self, node, r):
       
        results = []
        if node == None:
            return results
        if node.left == None and node.right == None:
            
            if r['xmin'] <= node.x and r['xmax'] >= node.x and r['ymin'] <= node.y and r['ymax'] >= node.y:
                results.append(node)
         
        else:
            if r['xmin'] <= node.x and r['xmax'] >= node.x and r['ymin'] <= node.y and r['ymax'] >= node.y:
                results.append(node)
                
            if self.is_fully_contained(node.left, r):
                results += self.traverse(node.left)
            
            elif self.is_intersect(node.left, r):
                results += self.range_search(node.left, r)
            
            if self.is_fully_contained(node.right, r):
                results += self.traverse(node.right)
                
            elif self.is_intersect(node.right, r):
                results += self.range_search(node.right, r)
               
        return results

    def traverse(self, node):
        members = []
        if node:
            members += self.traverse(node.left)
            members.append(node)
            members += self.traverse(node.right)
        return members

In [186]:
def min_max_distance(cluster1, cluster2):
    #start = timeit.default_timer()
    Z =distance.cdist(cluster1, cluster2, 'euclidean')
    #print(Z.min()**2,Z.max()**2)
    #stop = timeit.default_timer()
    #print('Time for mmr: ', stop - start)

    return Z.min()**2,Z.max()**2


    mindis = float('inf')
    maxdis = -1
    for i in cluster1:
        for j in cluster2:
            dis = distance_euclidean(i, j)
            if mindis > dis:
                mindis = dis
                item = (i, j)
            if maxdis < dis:
                maxdis = dis
    return mindis,maxdis

In [187]:
#gen Dataset

def latlngToGlobalXY(coords):
    radius = 6371.0
    dlon = math.pi*(coords[0]-lonMean)/180.0
    dlat = math.pi*(coords[1]-latMean)/180.0
    x = radius*dlon*math.cos(math.pi*latMean/180.0)
    y = radius*dlat
    return x,y
df_s = pd.read_json('estaciones.json', orient='columns')
stations_positions = df_s[['lon','lat']].values.reshape(-1,2)
lonMean = np.average(stations_positions[:,0])
latMean = np.average(stations_positions[:,1])
locs_bici = np.apply_along_axis(latlngToGlobalXY, 1, stations_positions)
X = locs_bici[:,0]
Y = locs_bici[:,1]

In [167]:
height = 4
kdt = kdTree(X,Y,height)

In [168]:
nodes = kdt.traverse(kdt.root)

In [169]:
nodes

In [170]:
def createLevelMatrix():
    nodeId = []
    for i in range(0,len(nodes)):
        nodeId.append(-1)
    levelMatrix = np.empty(
                shape=(height + 1, 2 ** height + 1), dtype=kdTree_node)
    for i in range(0,len(nodes)):
        nodeId[nodes[i].height] = nodeId[nodes[i].height] + 1
        #print(nodes[i].height,":",nodeId[nodes[i].height])
        levelMatrix[nodes[i].height][nodeId[nodes[i].height]] = nodes[i]
        nodes[i].id = nodeId[nodes[i].height]
        l = len(nodes[i].points)
        #print(nodes[i].height,l)
    return levelMatrix

In [171]:
levelMatrix = createLevelMatrix()

In [209]:
def createDistanceMatrix():
    dismatrix = np.empty(
        shape=(height + 1, 2 ** height + 1,
               2 ** height + 1),
        dtype=tuple)
    for l in range(0,height):
            for i in range(0,2**l ):
                for j in range(0,2**l ):
                    if (len(levelMatrix[l][i].points) == 0 or len(levelMatrix[l][j].points) == 0):
                        return dismatrix
                    dismatrix[l, i, j] = min_max_distance(levelMatrix[l][i].points, levelMatrix[l][j].points)
                    print(min_max_distance(levelMatrix[l][i].points, levelMatrix[l][j].points))
    return dismatrix

In [210]:
dismatrix = createDistanceMatrix()

(0.0, 114.79018678381388)
(0.0, 91.21034872331312)
(0.0, 114.79018678381388)
(0.0, 114.79018678381388)
(0.0, 91.10772370011055)
(0.0, 91.21034872331312)
(0.0, 91.21034872331312)
(0.0, 101.53689425566371)
(0.0, 87.34121698833464)
(0.0, 91.21034872331312)
(0.0, 24.93678881525933)
(0.0, 101.53689425566371)
(0.0, 74.38559027104208)
(0.0, 101.53689425566371)
(0.0, 101.53689425566371)
(0.0, 82.65984804127747)
(0.0, 89.39576635270109)
(0.0, 87.34121698833464)
(0.0, 74.38559027104208)
(0.0, 89.39576635270109)
(0.0, 19.787161697447708)
(0.0, 79.88868584788663)
(0.0, 91.21034872331312)
(0.0, 79.88868584788663)
(0.0, 79.00874000192181)
(0.5251835970051195, 101.53689425566371)
